## Notebook pour la Résolution de Sudoku avec Modèles Graphiques Probabilistes

Ce notebook présentera une approche pour résoudre des puzzles de Sudoku en utilisant la programmation probabiliste avec la bibliothèque `Infer.NET`. Nous explorerons d'abord une solution naïve, puis une solution plus sophistiquée et robuste.

### 1. Introduction à la Programmation Probabiliste

La programmation probabiliste permet de représenter des problèmes complexes en utilisant des modèles graphiques où les variables aléatoires sont interconnectées par des probabilités conditionnelles. Pour le Sudoku, chaque cellule de la grille peut être vue comme une variable aléatoire avec des probabilités associées aux valeurs possibles (1 à 9). Les contraintes du Sudoku (chaque chiffre doit apparaître une fois par ligne, colonne et boîte 3x3) sont incorporées dans ce modèle probabiliste.

**Références :**
- [Introduction à Infer.NET](https://dotnet.github.io/infer/)
- [Tutoriel d'inférence probabiliste](https://dotnet.github.io/infer/InferNet101.pdf)

### 2. Configuration de l'environnement

Installez les packages nécessaires pour ce notebook :

In [9]:
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

Installed Packages Microsoft.ML.Probabilistic, 0.4.2403.801 Microsoft.ML.Probabilistic.Compiler, 0.4.2403.801

### 3. Importation des Classes de Base

Nous allons importer les classes de base définies dans le notebook précédent, fournissant notamment la représentation, le chargement et l'affichage de Sudokus, et l'infrastructure de résolution.

In [10]:
#!import Sudoku-0-Environment.ipynb

# Notebook 0: Classes de Base pour la Résolution de Sudoku

Ce notebook contient les classes de base nécessaires pour la manipulation et la résolution des grilles de Sudoku. Il sera importé dans les autres notebooks pour réutiliser ces classes.

## Importation des Bibliothèques Nécessaires

Nous commençons par importer les bibliothèques nécessaires.


Installed Packages XPlot.Plotly.Interactive, 4.1.0

## Définition de la classe SudokuGrid

Nous définissons ici la classe SudokuGrid qui représente une grille de Sudoku et fournit des méthodes pour manipuler et afficher les grilles.


## Définition de l'interface ISudokuSolver

Nous définissons ici l'interface ISudokuSolver qui sera implémentée par les différentes stratégies de résolution de Sudoku.


## Définition de la classe SudokuHelper

Nous ajoutons ici la classe SudokuHelper qui contient des méthodes utilitaires pour charger  des grilles de Sudoku et tester des solvers.

- `GetSudokus` : Renvoie des listes de Sudoku issues de fichiers de 3 difficultés différentes.
- `SolveSudoku` : effectue un test simple d'un solver sur un sudoku donné.
- `TestSolvers` : exécute les tests de performance sur plusieurs solveurs.
- `DisplayResults` : affiche les résultats des tests sous forme de graphiques.



### 4. Implémentation du Solver Naïf

Nous commencerons par implémenter un solver naïf en utilisant `Infer.NET`.

#### Principe du Solver Naïf

Le solver naïf initialise chaque cellule de la grille de Sudoku avec une distribution uniforme sur les valeurs possibles (1 à 9) et ajoute des contraintes pour garantir que les valeurs dans chaque ligne, colonne et boîte sont distinctes. 

In [11]:
// Importation des bibliothèques nécessaires
using System.Linq;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Models.Attributes;
using System.Collections.Generic;

public class NaiveProbabilisticSolver : ISudokuSolver
{
    private static NaiveSudokuModel naiveModel = new NaiveSudokuModel();
    
    public SudokuGrid Solve(SudokuGrid s)
    {
        var toReturn = (SudokuGrid) s.Clone();
        naiveModel.SolveSudoku(toReturn);
        return toReturn;
    }
}

public class NaiveSudokuModel
{
    private static List<int> CellDomain = Enumerable.Range(1, 9).ToList();
    private static List<int> CellIndices = Enumerable.Range(0, 81).ToList();
    
    public virtual void SolveSudoku(SudokuGrid s)
    {
        var algo = new ExpectationPropagation();
        var engine = new InferenceEngine(algo);
        // engine.ShowFactorGraph = true;
        
        // Implémentation naïve: une variable aléatoire entière par cellule
        var cells = new List<Variable<int>>(CellIndices.Count);
        foreach (var cellIndex in CellIndices)
        {
            // On initialise le vecteur de probabilités de façon uniforme pour les chiffres de 1 à 9
            var baseProbas = Enumerable.Repeat(1.0, CellDomain.Count).ToList();
            
            // Création et ajout de la variable aléatoire
            var cell = Variable.Discrete(baseProbas.ToArray());
            cells.Add(cell);
        }
        
        // Ajout des contraintes de Sudoku (all diff pour tous les voisinages)
        foreach (var cellIndex in CellIndices)
        {
            foreach (var neighbourCellIndex in SudokuGrid.CellNeighbours[cellIndex/9][cellIndex%9])
            {
                var oneDIndex = neighbourCellIndex.row * 9 + neighbourCellIndex.column;
                
                // On ajoute la contrainte une seule fois par paire de cellules
                if (oneDIndex > cellIndex)
                {
                    Variable.ConstrainFalse(cells[cellIndex] == cells[oneDIndex]);
                }
            }
        }
        
        // On affecte les valeurs fournies par le masque à résoudre comme variables observées
        foreach (var cellIndex in CellIndices)
        {
            if (s.Cells[cellIndex / 9, cellIndex % 9] > 0)
            {
                cells[cellIndex].ObservedValue = s.Cells[cellIndex / 9, cellIndex % 9] - 1;
            }
        }
        
        // On infère les valeurs des cellules non observées
        foreach (var cellIndex in CellIndices)
        {
            if (s.Cells[cellIndex / 9, cellIndex % 9] == 0)
            {
                var result = (Discrete)engine.Infer(cells[cellIndex]);
                // On met à jour la grille avec la valeur inférée
                s.Cells[cellIndex / 9, cellIndex % 9] = result.Point + 1;
            }
        }
    }
}


#### Test du solver naïf sur 2 sudokus simples

In [12]:
var easySudokus = SudokuHelper.GetSudokus(SudokuDifficulty.Easy).Take(2).ToList();
var naiveSolver = new NaiveProbabilisticSolver();

foreach (var sudoku in easySudokus)
{
    SudokuHelper.SolveSudoku(sudoku, naiveSolver);
}

Résolution par le solver NaiveProbabilisticSolver du Sudoku:
 -------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50


Sudoku renvoyé:
-------------------------------
| 9  6  2 | 1  8  5 | 4  7  3 | 
| 1  7  4 | 9  6  3 | 8  2  5 | 
| 5  3  8 | 4  2  7 | 1  6  9 | 
-------------------------------
| 8  2  6 | 3  5  9 | 7  4  1 | 
| 3  5  7 | 8  1  4 | 2  9  6 | 
| 4  9  1 | 6  7  2 | 5  3  8 | 
-------------------------------
| 2  4  9 | 5  3  8 | 6  1  7 | 
| 7  1  5 | 2  9  6 | 3  8  4 | 
| 6  8  3 | 7  4  1 | 9  5  2 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 13291,3637 ms

Résolution par le solver NaiveProbabilisticSolver du Sudoku:
 -------------------------------
|       3 |    2    | 6       | 
| 9       | 3     5 |       1 | 
|       1 | 8     6 | 4       | 
-------------------------------
|       8 | 1     2 | 9       | 
| 7       |         |       8 | 
|       6 | 7     8 | 2       | 
-------------------------------
|       2 | 6     9 | 5       | 
| 8       | 2     3 |       9 | 
|       5 |    1    | 3       | 
-------------------------------

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50


Sudoku renvoyé:
-------------------------------
| 4  8  3 | 9  2  1 | 6  5  7 | 
| 9  6  7 | 3  4  5 | 8  2  1 | 
| 2  5  1 | 8  7  6 | 4  9  3 | 
-------------------------------
| 5  4  8 | 1  3  2 | 9  7  6 | 
| 7  2  9 | 5  6  4 | 1  3  8 | 
| 1  3  6 | 7  9  8 | 2  4  5 | 
-------------------------------
| 3  7  2 | 6  8  9 | 5  1  4 | 
| 8  1  4 | 2  5  3 | 7  6  9 | 
| 6  9  5 | 4  1  7 | 3  8  2 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 29898,0202 ms

On constate que le solver naïf a besoin de recompiler un nouveau modèle à chaque nouvelle résolution.
Nous pouvons implémenter un solver plus robuste qui ne nécessitera pas de nouvelle compilation, par l'introduction de nouvelles variables aléatoires.

### 5. Implémentation du Solver Robuste

Le solver robuste améliore la solution naïve en utilisant des distributions de Dirichlet pour modéliser les probabilités des valeurs possibles pour chaque cellule. Ce modèle permet d'initialiser les valeurs avec des probabilités non uniformes et de réutiliser les informations d'un Sudoku à l'autre sans recompilation complète.

#### Principe du Solver Robuste

Le solver robuste utilise des distributions de Dirichlet pour chaque cellule, représentant les probabilités des valeurs possibles. Les contraintes de Sudoku sont ajoutées pour garantir que les valeurs sont distinctes dans chaque ligne, colonne et boîte.

In [13]:
using System.Linq;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Models.Attributes;
using System.Collections.Generic;
using Range = Microsoft.ML.Probabilistic.Models.Range;

public class RobustProbabilisticSolver : ISudokuSolver
{
    private static RobustSudokuModel robustModel = new RobustSudokuModel();    
    
    public SudokuGrid Solve(SudokuGrid s)
    {
        var toReturn = (SudokuGrid) s.Clone();
        robustModel.SolveSudoku(toReturn);
        return toReturn;
    }

}

public class RobustSudokuModel
{
    // Moteur d'inférence
    public InferenceEngine InferenceEngine;
    
    // Domaine des valeurs possibles pour chaque cellule
    public static List<int> CellDomain = Enumerable.Range(1, 9).ToList();
    
    // Indices des cellules
    public static List<int> CellIndices = Enumerable.Range(0, 81).ToList();
    
    // Distribution a priori des cellules
    public VariableArray<Dirichlet> CellsPrior;
    
    // Probabilités des valeurs possibles pour chaque cellule
    public VariableArray<Vector> ProbCells;
    
    // Valeurs des cellules
    public VariableArray<int> Cells;
    
    // Epsilon pour les probabilités
    public const double EpsilonProba = 0.00000001;
    
    // Probabilité fixe pour une valeur donnée
    public static double FixedValueProba = 1.0 - ((CellDomain.Count - 1) * EpsilonProba);
    
    public RobustSudokuModel()
    {
        // Création des ranges pour les valeurs et les cellules
        Range valuesRange = new Range(CellDomain.Count).Named("valuesRange");
        Range cellsRange = new Range(CellIndices.Count).Named("cellsRange");
        

        // Cf https://en.wikipedia.org/wiki/Categorical_distribution et https://en.wikipedia.org/wiki/Categorical_distribution#Bayesian_inference_using_conjugate_prior pour le choix des distributions
        // et le chapitre 6 de https://dotnet.github.io/infer/InferNet101.pdf pour l'implémentation dans Infer.Net


        // Création des variables a priori pour les probabilités des cellules
        CellsPrior = Variable.Array<Dirichlet>(cellsRange).Named("CellsPrior");
        
        // Création des variables pour les probabilités des valeurs possibles pour chaque cellule
        ProbCells = Variable.Array<Vector>(cellsRange).Named("ProbCells");
        ProbCells[cellsRange] = Variable<Vector>.Random(CellsPrior[cellsRange]);
        ProbCells.SetValueRange(valuesRange);
        
        // Initialisation des distributions uniformes pour les probabilités a priori
        Dirichlet[] dirUnifArray = Enumerable.Repeat(Dirichlet.Uniform(CellDomain.Count), CellIndices.Count).ToArray();
        CellsPrior.ObservedValue = dirUnifArray;
        
        // Création des variables pour les valeurs des cellules
        Cells = Variable.Array<int>(cellsRange);
        Cells[cellsRange] = Variable.Discrete(ProbCells[cellsRange]);
        
        // Ajout des contraintes de Sudoku (all diff pour tous les voisinages)
        foreach (var cellIndex in CellIndices)
        {
            foreach (var neighbourCellIndex in SudokuGrid.CellNeighbours[cellIndex/9][cellIndex%9])
            {
                var oneDIndex = neighbourCellIndex.row * 9 + neighbourCellIndex.column;
                if (oneDIndex > cellIndex)
                {
                    Variable.ConstrainFalse(Cells[cellIndex] == Cells[oneDIndex]);
                }
            }
        }
        
        // Création du moteur d'inférence
        IAlgorithm algo = new ExpectationPropagation();
        algo.DefaultNumberOfIterations = 50;
        InferenceEngine = new InferenceEngine(algo);
    }
    
    public virtual void SolveSudoku(SudokuGrid s)
    {
        // Création des distributions uniformes pour les probabilités a priori
        Dirichlet[] dirArray = Enumerable.Repeat(Dirichlet.Uniform(CellDomain.Count), CellIndices.Count).ToArray();
        
        // Affectation des valeurs fournies par le masque à résoudre comme valeurs fixes
        foreach (var cellIndex in CellIndices)
        {
            if (s.Cells[cellIndex / 9, cellIndex % 9] > 0)
            {
                Vector v = Vector.Constant(CellDomain.Count, EpsilonProba);
                v[s.Cells[cellIndex / 9, cellIndex % 9] - 1] = FixedValueProba;
                dirArray[cellIndex] = Dirichlet.PointMass(v);
            }
        }
        
        // Affectation des distributions a priori des cellules
        CellsPrior.ObservedValue = dirArray;
        
        // Inférence des probabilités des valeurs possibles pour chaque cellule
        DoInference(dirArray, s.Cells);


       
    }


    protected virtual void DoInference(Dirichlet[] dirArray, int[,] sudokuCells)
    {
        // Todo: tester en inférant sur d'autres variables aléatoire,
        // et/ou en ayant une approche itérative: On conserve uniquement les cellules dont les valeurs ont les meilleures probabilités 
        //et on réinjecte ces valeurs dans CellsPrior comme c'est également fait dans le projet neural nets. 
        //

        // IFunction draw_categorical(n)// where n is the number of samples to draw from the categorical distribution
        // {
        //
        // r = 1

        /* for (i=0; i<9; i++)
            for (j=0; j<9; j++)
	            for (k=0; k<9; k++)
	    	        ps[i][j][k] = probs[i][j][k].p; */


        //DistributionRefArray<Discrete, int> cellsPosterior = (DistributionRefArray<Discrete, int>)InferenceEngine.Infer(Cells);
        //var cellValues = cellsPosterior.Point.Select(i => i + 1).ToList();

        //Autre possibilité de variable d'inférence (bis)
        Dirichlet[] cellsProbsPosterior = InferenceEngine.Infer<Dirichlet[]>(ProbCells);

        foreach (var cellIndex in CellIndices)
        {
            if (sudokuCells[cellIndex/9, cellIndex%9] == 0)
            {
                //s.Cellules[cellIndex] = cellValues[cellIndex];

                var mode = cellsProbsPosterior[cellIndex].GetMode();
                var value = mode.IndexOf(mode.Max()) + 1;
                sudokuCells[cellIndex/9, cellIndex%9] = value;
            }
        }
    }

}

### 6. Test de la Résolution 

Nous allons tester les deux solveurs (`NaiveProbabilisticSolver` et `RobustProbabilisticSolver`) sur quelques grilles de Sudoku faciles.

#### Chargement de Sudokus Faciles et Résolution


In [14]:
// Définir les solveurs à tester
var solvers = new List<(string Name, ISudokuSolver Solver)>
{
    // ("NaiveProbabilisticSolver", new NaiveProbabilisticSolver()),
    ("RobustProbabilisticSolver", new RobustProbabilisticSolver())
};

display("Test des sudokus faciles");

// Charger quelques grilles de Sudoku faciles
var easySudokus = SudokuHelper.GetSudokus(SudokuDifficulty.Easy).Take(2).ToList();

// Résoudre et afficher les résultats pour chaque solver et chaque grille


foreach (var solver in solvers)
{
    
    foreach (var sudoku in easySudokus)
    {
        var solvedSudoku = SudokuHelper.SolveSudoku(sudoku, solver.Solver);
    }

}



Test des sudokus faciles

Résolution par le solver RobustProbabilisticSolver du Sudoku:
 -------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50


Sudoku renvoyé:
-------------------------------
| 9  6  2 | 1  8  5 | 4  7  3 | 
| 1  7  4 | 9  6  3 | 8  2  5 | 
| 5  3  8 | 4  2  7 | 1  6  9 | 
-------------------------------
| 8  2  6 | 3  5  9 | 7  4  1 | 
| 3  5  7 | 8  1  4 | 2  9  6 | 
| 4  9  1 | 6  7  2 | 5  3  8 | 
-------------------------------
| 2  4  9 | 5  3  8 | 6  1  7 | 
| 7  1  5 | 2  9  6 | 3  8  4 | 
| 6  8  3 | 7  4  1 | 9  5  2 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 276223,6046 ms

Résolution par le solver RobustProbabilisticSolver du Sudoku:
 -------------------------------
|       3 |    2    | 6       | 
| 9       | 3     5 |       1 | 
|       1 | 8     6 | 4       | 
-------------------------------
|       8 | 1     2 | 9       | 
| 7       |         |       8 | 
|       6 | 7     8 | 2       | 
-------------------------------
|       2 | 6     9 | 5       | 
| 8       | 2     3 |       9 | 
|       5 |    1    | 3       | 
-------------------------------

Iterating: 
.........|.........|.........|.........|.........| 50


Sudoku renvoyé:
-------------------------------
| 4  8  3 | 9  2  1 | 6  5  7 | 
| 9  6  7 | 3  4  5 | 8  2  1 | 
| 2  5  1 | 8  7  6 | 4  9  3 | 
-------------------------------
| 5  4  8 | 1  3  2 | 9  7  6 | 
| 7  2  9 | 5  6  4 | 1  3  8 | 
| 1  3  6 | 7  9  8 | 2  4  5 | 
-------------------------------
| 3  7  2 | 6  8  9 | 5  1  4 | 
| 8  1  4 | 2  5  3 | 7  6  9 | 
| 6  9  5 | 4  1  7 | 3  8  2 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 63,0568 ms

On constate qu'une fois le modèle compilé, le solver robuste peut effectuer de nouvelles inférences dans un temps très raisonnable.

#### Tests avec un Sudoku Medium

In [15]:
display("Test des sudokus medium");


var mediumSudokus = SudokuHelper.GetSudokus(SudokuDifficulty.Medium).Take(1).ToList();
foreach (var solver in solvers)
{
    
    foreach (var sudoku in mediumSudokus)
    {
        var solvedSudoku = SudokuHelper.SolveSudoku(sudoku, solver.Solver);
    }

}

Test des sudokus medium

Résolution par le solver RobustProbabilisticSolver du Sudoku:
 -------------------------------
| 8  5    |       2 | 4       | 
| 7  2    |         |       9 | 
|       4 |         |         | 
-------------------------------
|         | 1     7 |       2 | 
| 3     5 |         | 9       | 
|    4    |         |         | 
-------------------------------
|         |    8    |    7    | 
|    1  7 |         |         | 
|         |    3  6 |    4    | 
-------------------------------

Iterating: 
.........|.........|.........|.........|.........| 50


Sudoku renvoyé:
-------------------------------
| 8  5  1 | 9  9  2 | 4  3  7 | 
| 7  2  6 | 3  4  4 | 8  6  9 | 
| 9  6  4 | 5  1  8 | 1  2  7 | 
-------------------------------
| 9  8  9 | 1  6  7 | 3  5  2 | 
| 3  7  5 | 8  2  4 | 9  1  4 | 
| 1  4  2 | 3  9  3 | 7  1  7 | 
-------------------------------
| 4  3  3 | 5  8  1 | 2  7  5 | 
| 4  1  7 | 2  9  5 | 6  9  3 | 
| 5  9  9 | 7  3  6 | 1  4  8 | 
-------------------------------
Nombre d'erreurs réstantes: 37
Temps de résolution: 36,9977 ms

#### Conclusion sur la Résolution des Sudokus de Difficulté Moyenne

Les solveurs probabilistes `NaiveProbabilisticSolver` et `RobustProbabilisticSolver` n'ont pas réussi à résoudre les Sudokus de difficulté moyenne. Les solveurs probabilistes actuels montrent une bonne performance sur les grilles faciles mais échouent sur les grilles plus complexes. Cette limitation met en évidence le besoin d'améliorer les modèles probabilistes, notamment en utilisant des techniques d'inférence itérative.

### 7. Implémentation du Solver Itératif

Nous allons maintenant implémenter un solver itératif basé sur le modèle robuste. Ce solver utilise une approche itérative pour améliorer les performances sur des grilles de Sudoku plus complexes.

#### Principe du Solver Itératif

Le solver itératif améliore le modèle robuste en itérant sur les cellules les plus probables à chaque étape et en réinjectant les valeurs inférées dans les distributions a priori. Cette approche permet de raffiner progressivement les valeurs des cellules jusqu'à ce que toutes les cellules soient résolues.

#### Code du Solver Itératif

In [16]:
using System;
using System.Linq;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using System.Collections.Generic;

public class IterativeSudokuModel : RobustSudokuModel
{
    public int NbIterationCells { get; set; } = 2;

    protected override void DoInference(Dirichlet[] dirArray, int[,] sudokuCells)
    {
        int cellDiscovered = CountNonZeroElements(sudokuCells);

        // Iteration tant que l'on a pas découvert toutes les cases
        while (cellDiscovered < CellIndices.Count)
        {
            Dirichlet[] cellsProbsPosterior = InferenceEngine.Infer<Dirichlet[]>(ProbCells);

            int[] bestCellsProbsPosteriorIndex = GetBestDirichletSubArrayIndex(cellsProbsPosterior, NbIterationCells, sudokuCells);

            foreach (var index in bestCellsProbsPosteriorIndex)
            {
                var mode = cellsProbsPosterior[index].GetMode();
                var value = mode.IndexOf(mode.Max()) + 1;

                Vector v = Vector.Constant(CellDomain.Count, EpsilonProba);
                v[value - 1] = FixedValueProba;

                dirArray[index] = Dirichlet.PointMass(v);

                if (sudokuCells[index / 9, index % 9] == 0)
                    cellDiscovered++;
                sudokuCells[index / 9, index % 9] = value;
            }

            CellsPrior.ObservedValue = dirArray;
        }
    }

    private int[] GetBestDirichletSubArrayIndex(Dirichlet[] dirichletArray, int N, int[,] sudokuCells)
    {
        // Initialise la liste des N meilleurs index avec les N premiers index de dirichletArray pour les cellules vides
        var emptyCells = sudokuCells
            .Cast<int>()
            .Select((cell, index) => new { cell, index })
            .Where(x => x.cell == 0)
            .Select(x => x.index)
            .Take(N)
            .ToArray();

        // Pour chaque cellule == 0 du sudoku
        foreach (var cellIndex in CellIndices)
        {
            if (sudokuCells[cellIndex / 9, cellIndex % 9] == 0)
            {
                var currentMode = dirichletArray[cellIndex].GetMode();

                int minDirIndex = emptyCells[0];

                // Récupère l'index du Dirichlet le plus petit de la liste d'index des meilleurs Dirichlet
                foreach (var index in emptyCells)
                {
                    var currentDirMode = dirichletArray[index].GetMode();
                    var minDirMode = dirichletArray[minDirIndex].GetMode();

                    if (currentDirMode.Max() < minDirMode.Max())
                    {
                        minDirIndex = index;
                    }
                }
                // Remplace ce Dirichlet si la valeur max du Dirichlet de la cellule actuelle est supérieure
                if (dirichletArray[minDirIndex].GetMode().Max() < currentMode.Max())
                {
                    emptyCells[Array.IndexOf(emptyCells, minDirIndex)] = cellIndex;
                }
            }
        }
        return emptyCells;
    }

    private int CountNonZeroElements(int[,] array)
    {
        int count = 0;
        foreach (var element in array)
        {
            if (element > 0)
            {
                count++;
            }
        }
        return count;
    }
}

public class IterativeProbabilisticSolver : ISudokuSolver
{
    public static IterativeSudokuModel Model = new IterativeSudokuModel();    
    
    public SudokuGrid Solve(SudokuGrid s)
    {
        var toReturn = (SudokuGrid) s.Clone();
        Model.SolveSudoku(toReturn);
        return toReturn;
    }
}


#### Test sur un Sudoku simple

In [17]:
// Tester le solver itératif sur un Sudoku de difficulté facile
var iterativeSolver = new IterativeProbabilisticSolver();
var easySudokus = SudokuHelper.GetSudokus(SudokuDifficulty.Easy).Take(2).ToList();

    
foreach (var sudoku in easySudokus)
{
    var solvedSudoku = SudokuHelper.SolveSudoku(sudoku, iterativeSolver);
}


Résolution par le solver IterativeProbabilisticSolver du Sudoku:
 -------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|

Sudoku renvoyé:
-------------------------------
| 9  6  2 | 1  8  5 | 4  7  3 | 
| 1  7  4 | 9  6  3 | 8  2  5 | 
| 5  3  8 | 4  2  7 | 1  6  9 | 
-------------------------------
| 8  2  6 | 3  5  9 | 7  4  1 | 
| 3  5  7 | 8  1  4 | 2  9  6 | 
| 4  9  1 | 6  7  2 | 5  3  8 | 
-------------------------------
| 2  4  9 | 5  3  8 | 6  1  7 | 
| 7  1  5 | 2  9  6 | 3  8  4 | 
| 6  8  3 | 7  4  1 | 9  5  2 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 312657,4218 ms

Résolution par le solver IterativeProbabilisticSolver du Sudoku:
 -------------------------------
|       3 |    2    | 6       | 
| 9       | 3     5 |       1 | 
|       1 | 8     6 | 4       | 
-------------------------------
|       8 | 1     2 | 9       | 
| 7       |         |       8 | 
|       6 | 7     8 | 2       | 
-------------------------------
|       2 | 6     9 | 5       | 
| 8       | 2     3 |       9 | 
|       5 |    1    | 3       | 
-------------------------------

Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating:

Sudoku renvoyé:
-------------------------------
| 4  8  3 | 9  2  1 | 6  5  7 | 
| 9  6  7 | 3  4  5 | 8  2  1 | 
| 2  5  1 | 8  7  6 | 4  9  3 | 
-------------------------------
| 5  4  8 | 1  3  2 | 9  7  6 | 
| 7  2  9 | 5  6  4 | 1  3  8 | 
| 1  3  6 | 7  9  8 | 2  4  5 | 
-------------------------------
| 3  7  2 | 6  8  9 | 5  1  4 | 
| 8  1  4 | 2  5  3 | 7  6  9 | 
| 6  9  5 | 4  1  7 | 3  8  2 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 1474,015 ms

#### Test sur un Sudoku medium

In [18]:
// Tester le solver itératif sur un Sudoku de difficulté medium
IterativeProbabilisticSolver.Model.NbIterationCells = 1;
var mediumSudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Medium).Skip(1).First();
SudokuHelper.SolveSudoku(mediumSudoku, iterativeSolver);

Résolution par le solver IterativeProbabilisticSolver du Sudoku:
 -------------------------------
|       5 | 3       |         | 
| 8       |         |    2    | 
|    7    |    1    | 5       | 
-------------------------------
| 4       |       5 | 3       | 
|    1    |    7    |       6 | 
|       3 | 2       |    8    | 
-------------------------------
|    6    | 5       |       9 | 
|       4 |         |    3    | 
|         |       9 | 7       | 
-------------------------------

Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating:

Sudoku renvoyé:
-------------------------------
| 2  4  5 | 3  8  6 | 1  9  7 | 
| 8  3  1 | 9  5  7 | 6  2  4 | 
| 6  7  9 | 4  1  2 | 5  3  8 | 
-------------------------------
| 4  8  6 | 1  9  5 | 3  7  2 | 
| 9  1  2 | 8  7  3 | 4  5  6 | 
| 7  5  3 | 2  6  4 | 9  8  1 | 
-------------------------------
| 1  6  7 | 5  3  8 | 2  4  9 | 
| 5  9  4 | 7  2  1 | 8  3  3 | 
| 3  2  8 | 6  4  9 | 7  1  5 | 
-------------------------------
Nombre d'erreurs réstantes: 3
Temps de résolution: 3503,0365 ms

### 7. Utiliser une version précompilée

La version suivante récupère le modèle généré dans le répertoire GeneratedSource pour en faire une assembly compilée et économiser le temps conséquent de compilation.

In [19]:
#r "nuget: Microsoft.CodeAnalysis"
#r "nuget: Microsoft.CodeAnalysis.CSharp"

Installed Packages Microsoft.CodeAnalysis, 4.9.2 microsoft.codeanalysis.csharp, 2.10.0

In [20]:
using System;
using System.IO;
using System.Linq;
using System.Reflection;
using System.Threading;
using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Models.Attributes;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;
using Microsoft.ML.Probabilistic.Compiler.CodeModel;
using System.Collections.Generic;
using Microsoft.CodeAnalysis;
using Microsoft.CodeAnalysis.CSharp;
using Range = Microsoft.ML.Probabilistic.Models.Range;

public class PrecompiledRobustSudokuModel : ISudokuSolver
{
    private const string CompiledModelName = "RobustSudokuModel";
    private static string CompiledModelPath;

    private InferenceEngine InferenceEngine;
    private IGeneratedAlgorithm compiledModel;

    // Domaine des valeurs possibles pour chaque cellule
    private static List<int> CellDomain = Enumerable.Range(1, 9).ToList();

    // Indices des cellules
    private static List<int> CellIndices = Enumerable.Range(0, 81).ToList();

    // Distribution a priori des cellules
    private VariableArray<Dirichlet> CellsPrior;

    // Probabilités des valeurs possibles pour chaque cellule
    private VariableArray<Vector> ProbCells;

    // Valeurs des cellules
    private VariableArray<int> Cells;

    // Epsilon pour les probabilités
    private const double EpsilonProba = 0.00000001;

    // Probabilité fixe pour une valeur donnée
    private static double FixedValueProba = 1.0 - ((CellDomain.Count - 1) * EpsilonProba);

    static PrecompiledRobustSudokuModel()
    {
        CompiledModelPath = Path.Combine(Environment.CurrentDirectory, "CompiledModels");
    }

    public PrecompiledRobustSudokuModel()
    {
        if (LoadPrecompiledModel())
        {
            Console.WriteLine("Using precompiled model.");
        }
        else
        {
            Console.WriteLine("Loading or compiling model...");
            if (LoadAndCompileCsFile())
            {
                Console.WriteLine("Model compiled from .cs file.");
            }
            else
            {
                Console.WriteLine("Model compiled from scratch:");
                CompileModel();
            }
        }
    }

    private bool LoadPrecompiledModel()
    {
        string compiledFilePath = Path.Combine(CompiledModelPath, $"{CompiledModelName}.dll");
        display($"Compiled model Assembly path : {compiledFilePath}");
        if (File.Exists(compiledFilePath))
        {
            try
            {
                Assembly assembly = Assembly.LoadFrom(compiledFilePath);
                Type modelType = assembly.GetTypes().FirstOrDefault(t => typeof(IGeneratedAlgorithm).IsAssignableFrom(t));
                if (modelType != null)
                {
                    compiledModel = (IGeneratedAlgorithm)Activator.CreateInstance(modelType);
                    display($"Compiled model type: {modelType}");
                    return compiledModel != null;
                }
            }
            catch (IOException exc)
            {
                // Handle the case where the file is locked by another process
                Console.WriteLine("The compiled model DLL is currently in use by another process. Please close any other applications that might be using it and try again.");
                display(exc);
            }
        }
        return false;
    }

    private bool LoadAndCompileCsFile()
    {
        string csFilePath = Path.Combine(CompiledModelPath, $"{CompiledModelName}.cs");
        display($"Compiled model source path: {csFilePath}");
        if (File.Exists(csFilePath))
        {
            CompileCsToDll(csFilePath);
            return true;
        }
        return false;
    }

    private void CompileModel()
    {
        Range valuesRange = new Range(CellDomain.Count).Named("valuesRange");
        Range cellsRange = new Range(CellIndices.Count).Named("cellsRange");

        CellsPrior = Variable.Array<Dirichlet>(cellsRange).Named("CellsPrior");
        ProbCells = Variable.Array<Vector>(cellsRange).Named("ProbCells");
        ProbCells[cellsRange] = Variable<Vector>.Random(CellsPrior[cellsRange]);
        ProbCells.SetValueRange(valuesRange);

        Dirichlet[] dirUnifArray = Enumerable.Repeat(Dirichlet.Uniform(CellDomain.Count), CellIndices.Count).ToArray();
        CellsPrior.ObservedValue = dirUnifArray;

        Cells = Variable.Array<int>(cellsRange);
        Cells[cellsRange] = Variable.Discrete(ProbCells[cellsRange]);

        foreach (var cellIndex in CellIndices)
        {
            foreach (var neighbourCellIndex in SudokuGrid.CellNeighbours[cellIndex / 9][cellIndex % 9])
            {
                var oneDIndex = neighbourCellIndex.row * 9 + neighbourCellIndex.column;
                if (oneDIndex > cellIndex)
                {
                    Variable.ConstrainFalse(Cells[cellIndex] == Cells[oneDIndex]);
                }
            }
        }

        IAlgorithm algo = new ExpectationPropagation { DefaultNumberOfIterations = 50 };
        InferenceEngine = new InferenceEngine(algo);
        InferenceEngine.ShowProgress = false;

        compiledModel = InferenceEngine.GetCompiledInferenceAlgorithm(new IVariable[] { ProbCells, Cells });
        SaveCompiledModel();
    }

    private void SaveCompiledModel()
    {
        string generatedSourcePath = Path.Combine(Environment.CurrentDirectory, "GeneratedSource");
        display($"Generated source path: {generatedSourcePath}");

        var modelSourceFile = Directory.GetFiles(generatedSourcePath, "*.cs")
            .OrderByDescending(File.GetLastWriteTime)
            .FirstOrDefault();

        display($"Model source file: {modelSourceFile}");
        display($"Compiled model path: {CompiledModelPath}");

        if (modelSourceFile != null)
        {
            string compiledModelPath = Path.Combine(CompiledModelPath, $"{CompiledModelName}.cs");
            Directory.CreateDirectory(CompiledModelPath);
            File.Copy(modelSourceFile, compiledModelPath, true);
            CompileCsToDll(compiledModelPath);
        }
    }

    private void CompileCsToDll(string sourcePath)
    {
        string assemblyName = Path.Combine(CompiledModelPath, $"{CompiledModelName}.dll");
        var csharpCode = File.ReadAllText(sourcePath);

        var syntaxTree = CSharpSyntaxTree.ParseText(csharpCode);
        var references = AppDomain.CurrentDomain.GetAssemblies()
            .Where(a => !a.IsDynamic && !string.IsNullOrEmpty(a.Location))
            .Select(a => MetadataReference.CreateFromFile(a.Location))
            .Cast<MetadataReference>()
            .ToList();

        var compilation = CSharpCompilation.Create(
            Path.GetFileNameWithoutExtension(assemblyName),
            new[] { syntaxTree },
            references,
            new CSharpCompilationOptions(OutputKind.DynamicallyLinkedLibrary));

        var result = compilation.Emit(assemblyName);

        if (!result.Success)
        {
            var failures = result.Diagnostics.Where(diagnostic =>
                diagnostic.IsWarningAsError ||
                diagnostic.Severity == DiagnosticSeverity.Error);

            foreach (var diagnostic in failures)
            {
                Console.Error.WriteLine($"{diagnostic.Id}: {diagnostic.GetMessage()}");
            }
        }
    }

    public SudokuGrid Solve(SudokuGrid s)
    {
        if (compiledModel == null)
        {
            throw new InvalidOperationException("The compiled model is not loaded or initialized.");
        }

        var toReturn = (SudokuGrid)s.Clone();

        Dirichlet[] dirArray = Enumerable.Repeat(Dirichlet.Uniform(CellDomain.Count), CellIndices.Count).ToArray();

        foreach (var cellIndex in CellIndices)
        {
            if (s.Cells[cellIndex / 9, cellIndex % 9] > 0)
            {
                Vector v = Vector.Constant(CellDomain.Count, EpsilonProba);
                v[s.Cells[cellIndex / 9, cellIndex % 9] - 1] = FixedValueProba;
                dirArray[cellIndex] = Dirichlet.PointMass(v);
            }
        }

        display($"Setting observed value for CellsPrior with length {dirArray.Length}");
        compiledModel.SetObservedValue("CellsPrior", dirArray); // Set observed values in the compiled model
        compiledModel.Execute(50);

        Dirichlet[] cellsProbsPosterior = compiledModel.Marginal<Dirichlet[]>("ProbCells");

        foreach (var cellIndex in CellIndices)
        {
            if (toReturn.Cells[cellIndex / 9, cellIndex % 9] == 0)
            {
                var mode = cellsProbsPosterior[cellIndex].GetMode();
                var value = mode.IndexOf(mode.Max()) + 1;
                toReturn.Cells[cellIndex / 9, cellIndex % 9] = value;
            }
        }

        return toReturn;
    }
}




#### Tests

On teste le nouveau solver avec le code source archivé.

In [21]:
// Tester le solver avec modèle précompilé sur un Sudoku de difficulté facile
var precompiledSolver = new PrecompiledRobustSudokuModel();
var easySudokus = SudokuHelper.GetSudokus(SudokuDifficulty.Easy).Take(2).ToList();

foreach (var sudoku in easySudokus)
{
    var solvedSudoku = SudokuHelper.SolveSudoku(sudoku, precompiledSolver);
}

// Tester le solver avec modèle précompilé sur un Sudoku de difficulté moyenne
var mediumSudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Medium).Skip(1).First();
SudokuHelper.SolveSudoku(mediumSudoku, precompiledSolver);


Compiled model Assembly path : e:\Dev\AI\Cours\CoursIA\MyIA.AI.Notebooks\Sudoku\CompiledModels\RobustSudokuModel.dll

Compiled model type: Models.Model_EP

Using precompiled model.


Résolution par le solver PrecompiledRobustSudokuModel du Sudoku:
 -------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

Setting observed value for CellsPrior with length 81

...

Setting observed value for CellsPrior with length 81

Sudoku renvoyé:
-------------------------------
| 4  8  3 | 9  2  1 | 6  5  7 | 
| 9  6  7 | 3  4  5 | 8  2  1 | 
| 2  5  1 | 8  7  6 | 4  9  3 | 
-------------------------------
| 5  4  8 | 1  3  2 | 9  7  6 | 
| 7  2  9 | 5  6  4 | 1  3  8 | 
| 1  3  6 | 7  9  8 | 2  4  5 | 
-------------------------------
| 3  7  2 | 6  8  9 | 5  1  4 | 
| 8  1  4 | 2  5  3 | 7  6  9 | 
| 6  9  5 | 4  1  7 | 3  8  2 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 54,4291 ms

Résolution par le solver PrecompiledRobustSudokuModel du Sudoku:
 -------------------------------
|       5 | 3       |         | 
| 8       |         |    2    | 
|    7    |    1    | 5       | 
-------------------------------
| 4       |       5 | 3       | 
|    1    |    7    |       6 | 
|       3 | 2       |    8    | 
-------------------------------
|    6    | 5       |       9 | 
|       4 |         |    3    | 
|         |       9 | 7       | 
-------------------------------

Setting observed value for CellsPrior with length 81

Sudoku renvoyé:
-------------------------------
| 2  4  5 | 3  2  6 | 6  9  7 | 
| 8  3  1 | 7  5  7 | 6  2  3 | 
| 2  7  6 | 9  1  2 | 5  9  3 | 
-------------------------------
| 4  8  6 | 1  6  5 | 3  7  7 | 
| 9  1  2 | 8  7  3 | 9  5  6 | 
| 7  5  3 | 2  9  4 | 1  8  7 | 
-------------------------------
| 1  6  7 | 5  3  1 | 8  1  9 | 
| 7  9  4 | 7  6  1 | 6  3  2 | 
| 1  5  8 | 6  3  9 | 7  6  5 | 
-------------------------------
Nombre d'erreurs réstantes: 45
Temps de résolution: 44,0427 ms